# Doküman Yükleme ve Chunking

## 🎯 Bu Kodun Amacı
PDF dosyalarını yüklemek ve RAG için optimal boyutta parçalara ayırmak.

## Ne Yapacağız?
- **PyPDFLoader** ile "Attention is All You Need" makalesini yükleyeceğiz
- Her sayfanın içeriğini ve metadata'sını inceleyeceğiz
- **RecursiveCharacterTextSplitter** ile farklı chunking stratejileri test edeceğiz
- Üç farklı **chunk_size ve overlap** kombinasyonunu karşılaştıracağız
- En optimal stratejiyi (1000/100) seçip chunk örneklerini göreceğiz

## Çalıştırma
Kod çalıştırıldığında PDF yüklenir, farklı chunking stratejileri test edilir ve sonuçlar karşılaştırılır.

## 📄 PDF Dokümanını Yükleme

**Ne yapıyoruz?**
- "Attention is All You Need" makalesini PyPDFLoader ile yüklüyoruz
- Her sayfa ayrı bir Document objesi olarak yüklenir

**Document nedir?**
- `page_content`: Sayfanın metni
- `metadata`: Sayfa numarası, dosya adı gibi bilgiler

**Sonuç:** 15 sayfalık PDF → 15 ayrı Document

In [2]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

load_dotenv()

loader = PyPDFLoader("attention_is_all_you_need.pdf")
documents = loader.load()

print(f"Yüklenen sayfa sayısı: {len(documents)}\n")

Yüklenen sayfa sayısı: 15



## 👀 İlk Sayfa Önizlemesi

**Ne yapıyoruz?**
- Yüklenen dokümanın ilk sayfasını inceliyoruz
- Metadata bilgilerini görüntülüyoruz

**Metadata neden önemli?**
- RAG sisteminde "Bu bilgi 3. sayfada" gibi referans verebiliriz
- PDF hakkında (oluşturma tarihi, yaratıcı) bilgi içerir

**[:400]:** İlk 400 karakteri göster

In [ ]:
print("İLK SAYFA ÖNİZLEME:")
print("-" * 60)
print(documents[1].page_content[:400] + "...")
print(f"\nMetadata: {documents[0].metadata}\n")

İLK SAYFA ÖNİZLEME:
------------------------------------------------------------
Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.com
Noam Shazeer∗
Google Brain
noam@google.com
Niki Parmar∗
Google Research
nikip@google.com
Jakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
G...

Metadata: {'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-04-10T21:11:43+00:00', 'author': '', 'keywords': '', 'moddate': '2024-04-10T21:11:43+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'attention_is_all_you_need.pdf', 'total_pages': 15, 'page': 0, 'page_label': '1'}



## ✂️ Farklı Chunking Stratejilerini Test Etme

**Ne yapıyoruz?**
- 3 farklı chunk_size ve overlap kombinasyonunu test ediyoruz
- Hangisinin daha iyi sonuç verdiğini karşılaştırıyoruz

**Parametreler:**
- **chunk_size:** Her parçanın maksimum boyutu (karakter)
- **chunk_overlap:** Yan yana parçalar arasındaki örtüşme (karakter)

**Stratejiler:**
1. **500/50:** Küçük chunk'lar → Çok sayıda parça, spesifik bilgi
2. **1000/100:** Orta boy → Dengeli (EN POPÜLER) ✓
3. **1500/150:** Büyük chunk'lar → Az sayıda parça, geniş context

**Overlap neden önemli?**
- Cümleler chunk sınırında kesilirse anlam kaybolur
- Overlap ile önemli bilgiler birden fazla chunk'ta yer alır
- Örnek: "Transformer modelleri" iki chunk'ta da tam olmalı

**avg_length:** Ortalama chunk boyutu = (Toplam karakter) / (Chunk sayısı)

In [4]:
strategies = [
    {"chunk_size": 500, "chunk_overlap": 50},
    {"chunk_size": 1000, "chunk_overlap": 100},
    {"chunk_size": 1500, "chunk_overlap": 150}
]

print("CHUNKING STRATEJİLERİ:")
print("-" * 60)

for strategy in strategies:
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=strategy["chunk_size"],
        chunk_overlap=strategy["chunk_overlap"]
    )
    chunks = splitter.split_documents(documents)
    avg_length = sum(len(c.page_content) for c in chunks) / len(chunks)
    
    print(f"Size: {strategy['chunk_size']}, Overlap: {strategy['chunk_overlap']}")
    print(f"  Chunk sayısı: {len(chunks)}")
    print(f"  Ortalama uzunluk: {avg_length:.0f} karakter\n")

CHUNKING STRATEJİLERİ:
------------------------------------------------------------
Size: 500, Overlap: 50
  Chunk sayısı: 93
  Ortalama uzunluk: 433 karakter

Size: 1000, Overlap: 100
  Chunk sayısı: 49
  Ortalama uzunluk: 853 karakter

Size: 1500, Overlap: 150
  Chunk sayısı: 37
  Ortalama uzunluk: 1135 karakter



## 🎯 Optimal Stratejiyi Seçip Uygulama

**Ne yapıyoruz?**
- Test sonuçlarına göre **1000/100** stratejisini seçiyoruz
- Tüm dokümanı bu strateji ile chunk'lara bölüyoruz
- İlk 2 chunk'ı örnek olarak gösteriyoruz

**Neden 1000/100?**
- ✅ Dengeli chunk sayısı (49 chunk)
- ✅ Yeterli context (her chunk anlamlı bilgi içeriyor)
- ✅ Embedding modelleriyle uyumlu
- ✅ RAG performansı için ideal (hız + kalite)

**RecursiveCharacterTextSplitter neden "Recursive"?**
1. Önce paragraf sınırlarını (\n\n) dener
2. Yoksa satır sınırlarını (\n) dener
3. Yoksa cümle sınırlarını (. ! ?) dener
4. Son çare: kelime/karakter seviyesinde böler
→ **Anlamlı** bölme yapar!

**enumerate(final_chunks[:2], 1):** İlk 2 chunk'ı al, 1'den başlayarak numaralandır

**Sonraki Adım:** Bu chunk'lar S4_12'de embedding'lere çevrilip vektör veritabanına kaydedilecek.

In [5]:
final_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)
final_chunks = final_splitter.split_documents(documents)

print(f"SEÇİLEN STRATEJİ: 1000/100")
print(f"Toplam chunk: {len(final_chunks)}\n")

print("ÖRNEK CHUNK'LAR:")
print("-" * 60)
for i, chunk in enumerate(final_chunks[:2], 1):
    print(f"\nChunk {i} (Sayfa {chunk.metadata.get('page', 'N/A')}):")
    print(chunk.page_content[:200] + "...")

SEÇİLEN STRATEJİ: 1000/100
Toplam chunk: 49

ÖRNEK CHUNK'LAR:
------------------------------------------------------------

Chunk 1 (Sayfa 0):
Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
...

Chunk 2 (Sayfa 0):
based solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. Experiments on two machine translation tasks show these models to
be superior in quality while being more pa...
